In [1]:
import os
import glob
import numpy as np
import pandas as pd
os.chdir(r"C:\Users\cgree\Documents\Astra\Space_weather5_22\weakley_all")
import gzip
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy import stats

In [2]:
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

In [3]:
w = 'X:\\data_dumps\\6_28_21_ALL_OF_OMNI08_20'
df = pd.DataFrame()
df2 = pd.DataFrame()
list_of_files = os.listdir('X:\\data_dumps\\6_28_21_ALL_OF_OMNI08_20')
for x in list_of_files:
    df = pd.DataFrame()
    df = pd.read_csv(w + "\\" + x)
    df2 = df2.append(df)

In [4]:
w = 'X:\\data_dumps\\one_hour'
df8 = pd.DataFrame()
df9 = pd.DataFrame()
list_of_files = os.listdir('X:\\data_dumps\\one_hour')
for x in list_of_files:
    df8 = pd.DataFrame()
    df8 = pd.read_csv(w + "\\" + x)
    df9 = df9.append(df8)

In [11]:
df2["Date"] = pd.to_datetime(df2["EPOCH_TIME_yyyy-mm-ddThh:mm:ss.sssZ"], format="%Y-%m-%dT%H:%M:%S.%fZ")

In [5]:
df9["Date"] = pd.to_datetime(df9["TIME_AT_CENTER_OF_HOUR_yyyy-mm-ddThh:mm:ss.sssZ"], format="%Y-%m-%dT%H:%M:%S.%fZ")

In [12]:
# df2

In [13]:
df2.index = df2["Date"]
df9.index = df9["Date"]

In [14]:
del df2["Date"]
del df9['Date']

In [15]:
del df2["EPOCH_TIME_yyyy-mm-ddThh:mm:ss.sssZ"]
del df9['TIME_AT_CENTER_OF_HOUR_yyyy-mm-ddThh:mm:ss.sssZ']
# del df_new["M'SPH_CONTAMINATN_FLAG_(6=No"]
# del df_new["1=All"]
# del df_new["-1=n/a)"]

In [16]:
del df9['1AU_PROTONS>1_MEV_1/(SQcm-ster-s)']
del df9['1AU_PROTONS>2_MEV_1/(SQcm-ster-s)']
del df9['1AU_PROTONS>4_MEV_1/(SQcm-ster-s)']
del df9["M'SPH_CONTAMINATN_FLAG_(6=No"]
del df9['1=All']
del df9['-1=n/a)']
del df9['SOLAR_LYMAN-ALPHA_W/m^2']
del df9['PROTON_QI_']

In [17]:
df_new_hr = df9.copy()

In [18]:
list_of_files = os.listdir('X:\data_dumps\Supermag')
df7 = pd.DataFrame()
w = 'X:\data_dumps\Supermag'
for x in list_of_files:
    df6 = pd.DataFrame()
    df6 = pd.read_csv(w + "\\" + x)
    df7 = df7.append(df6)

In [19]:
df7 = df7.replace(to_replace= r'T', value= ' ', regex=True)

In [20]:
df7.set_index(pd.DatetimeIndex(df7['Date_UTC']), inplace=True)
del df7['Date_UTC']

In [21]:
df9 = df9.reset_index()
df7 = df7.reset_index()
df2 = df2.reset_index()
# del df2['index']
# del df7['index']
# del df9['index']

In [22]:
df2.set_index(pd.DatetimeIndex(df2['Date']), inplace=True)
del df2['Date']
df7.set_index(pd.DatetimeIndex(df7['Date_UTC']), inplace=True)
del df7['Date_UTC']
df9.set_index(pd.DatetimeIndex(df9['Date']), inplace=True)
del df9['Date']

In [23]:
dfx = df2.join(df7, how='inner')#  how='left')   #'inner')

In [24]:
df9 = df9.sort_index()

In [25]:
dfx=pd.merge_asof(dfx,df9, left_index=True, right_index=True)

In [27]:
dfx.columns

Index(['NO._IMF_AVG_', 'NO._IP_PLASMA_AVG_', 'MAG_AVG_B-VECTOR_nT',
       'BX__GSE_nT', 'BY__GSE_nT', 'BZ__GSE_nT', 'BY__GSM_nT', 'BZ__GSM_nT',
       'RMS_SD_B_SCALAR_nT', 'RMS_SD_FIELD_VECTOR_nT', 'FLOW_SPEED__GSE_km/s',
       'VX_VELOCITY__GSE_km/s', 'VY_VELOCITY__GSE_km/s',
       'VZ_VELOCITY__GSE_km/s', 'PROTON_DENSITY_n/cc', 'TEMPERATURE_K',
       'NA/NP_RATIO_', 'FLOW_PRESSURE_nPa', 'ELECTRIC_FIELD_mV/m',
       'PLASMA_BETA_', 'ALFVEN_MACH_NO._', '1AU_IP_MAGNETOSONIC_MACH_NO.__x',
       'X_(S/C)__GSE_Re', 'Y_(S/C)__GSE_Re', 'Z_(S/C)__GSE_Re',
       'X_(BSN)__GSE_Re', 'Y_(BSN)__GSE_Re', 'Z_(BSN)__GSE_Re', '1-M_AE_nT',
       '1-M_AL-INDEX_nT', 'AU-INDEX_nT_x', 'SYM/D_INDEX_nT', 'SYM/H_INDEX_nT',
       'ASY/D_INDEX_nT', 'ASY/H_INDEX_nT', 'Extent', 'IAGA', 'GEOLON',
       'GEOLAT', 'MLT', 'MCOLAT', 'IGRF_DECL', 'SZA', 'dbn_nez', 'dbe_nez',
       'dbz_nez', 'dbn_geo', 'dbe_geo', 'dbz_geo', 'MAGON', 'MAGLAT',
       'BARTEL_#_', '1AU_IP_MAG_AVG_B_nT', '1AU_IP_MAG_AVG_B-VECT

In [28]:
dfx['db_diff'] = ((dfx.dbn_nez.diff().pow(2) + (dfx.dbe_nez.diff().pow(2)))**(1/2))

In [34]:
features = dfx[['db_diff','DAILY_F10.7_',
'3-H_KP*10_',
'BY__GSE_nT',
'BZ__GSE_nT',
'1-H_AE_nT',
'SYM/H_INDEX_nT',
'RMS_SD_B_SCALAR_nT',
'RMS_SD_FIELD_VECTOR_nT',
'VY_VELOCITY__GSE_km/s',
'VZ_VELOCITY__GSE_km/s',
'PROTON_DENSITY_n/cc',
'FLOW_PRESSURE_nPa',
'ELECTRIC_FIELD_mV/m',
'PLASMA_BETA_',
'ALFVEN_MACH_NO._',
'AU-INDEX_nT_y',
'SYM/D_INDEX_nT',
'ASY/D_INDEX_nT',
'ASY/H_INDEX_nT',  
'RMSDEV_MAG_AVG_B_nT']]

In [35]:
features

,db_diff,DAILY_F10.7_,3-H_KP*10_,BY__GSE_nT,BZ__GSE_nT,1-H_AE_nT,SYM/H_INDEX_nT,RMS_SD_B_SCALAR_nT,RMS_SD_FIELD_VECTOR_nT,VY_VELOCITY__GSE_km/s,...,PROTON_DENSITY_n/cc,FLOW_PRESSURE_nPa,ELECTRIC_FIELD_mV/m,PLASMA_BETA_,ALFVEN_MACH_NO._,AU-INDEX_nT_y,SYM/D_INDEX_nT,ASY/D_INDEX_nT,ASY/H_INDEX_nT,RMSDEV_MAG_AVG_B_nT
2009-01-01 00:00:00,NaN,-7,59,0.60,-0.85,-33,-13,0.00,0.16,4.8,...,2.88,1.48,999.99,1.89,9.1,0.005982,0,13,12,0.3
2009-01-01 00:01:00,0.000000,-7,59,1.19,-0.91,-33,-12,0.07,1.14,10.5,...,2.89,1.48,0.28,1.87,9.0,0.005982,0,13,12,0.3
2009-01-01 00:02:00,0.000000,-7,59,1.87,-0.94,-33,-12,0.01,0.27,5.2,...,2.68,1.40,0.21,1.57,8.8,0.005982,0,14,12,0.3
2009-01-01 00:03:00,0.100000,-7,59,1.70,-1.05,-33,-12,0.00,0.00,4.2,...,2.66,1.39,0.29,1.52,8.7,0.005982,0,15,12,0.3
2009-01-01 00:04:00,0.000000,-7,59,1.71,-0.88,-33,-12,0.02,0.03,8.2,...,2.67,1.38,0.20,1.61,8.6,0.005982,0,15,12,0.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-30 23:56:00,0.316228,-4,9999,-2.31,-1.99,99999,-7,0.00,0.00,-3.2,...,2.50,1.02,1.13,1.83,11.0,0.006873,-1,17,13,0.2
2020-12-30 23:57:00,0.412311,-4,9999,-2.31,-1.99,99999,-7,0.00,0.00,-3.2,...,2.50,1.02,1.13,1.83,11.0,0.006873,0,16,13,0.2
2020-12-30 23:58:00,0.400000,-4,9999,9999.99,9999.99,99999,-7,9999.99,9999.99,-3.2,...,2.50,1.02,999.99,999.99,999.9,0.006873,0,16,12,0.2
2020-12-30 23:59:00,0.583095,-4,9999,-2.27,-1.52,99999,-7,0.00,0.00,99999.9,...,999.99,99.99,999.99,999.99,999.9,0.006873,-1,16,12,0.2


In [38]:
features.to_csv('X:\data_dumps\\10years.csv', index=True)